# Chapter 34: Intervals and Merging

> *"Intervals are everywhere—from meeting schedules to genomic sequences. Mastering interval manipulation is essential for solving a wide class of practical problems."* — Anonymous

---

## 34.1 Introduction to Intervals

An **interval** is a range defined by a start and an end point. Intervals are used to model time spans, ranges, segments, and many other real-world concepts. Common operations on intervals include:

- **Merging overlapping intervals** (e.g., combining meeting times)
- **Inserting a new interval into an existing set**
- **Checking if two intervals overlap**
- **Finding gaps or coverage**

Interval problems appear frequently in coding interviews and are often solved with sorting and greedy strategies.

### 34.1.1 Why Intervals Matter

```
┌─────────────────────────────────────────────────────────────────────┐
│                    IMPORTANCE OF INTERVALS                            │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. MEETING SCHEDULING: Determine if a person can attend all        │
│     meetings, or find the minimum number of meeting rooms.          │
│                                                                      │
│  2. CALENDAR APPLICATIONS: Merge overlapping appointments,          │
│     insert new events.                                               │
│                                                                      │
│  3. RESOURCE ALLOCATION: Allocate resources over time intervals.    │
│                                                                      │
│  4. GENOMICS: Intervals represent gene locations; merging and       │
│     overlapping operations are common.                              │
│                                                                      │
│  5. COMPUTER GRAPHICS: Interval coverage for scanline rendering.    │
│                                                                      │
│  6. NETWORK TRAFFIC: Analyze time windows of packet arrivals.       │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### 34.1.2 Common Representations

An interval is typically represented as a pair `[start, end]` (inclusive or exclusive). In this chapter, we'll assume inclusive intervals unless stated otherwise.

```python
# Using tuple (start, end) or a simple class
class Interval:
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def __repr__(self):
        return f"[{self.start}, {self.end}]"
```

---

## 34.2 Overlap Detection

Two intervals `[a, b]` and `[c, d]` overlap if they share any common point. For inclusive intervals, the condition is:

```
overlap = not (b < c or d < a)  # i.e., they are not separate
```

or more directly:

```python
def overlaps(i1, i2):
    return not (i1.end < i2.start or i2.end < i1.start)
```

If intervals are half‑open `[start, end)` (end exclusive), the condition is `i1.end > i2.start and i2.end > i1.start`.

---

## 34.3 Merging Overlapping Intervals (LeetCode 56)

**Problem:** Given an array of intervals where `intervals[i] = [start_i, end_i]`, merge all overlapping intervals and return the merged intervals.

**Idea:**
1. Sort intervals by start time.
2. Initialize a result list with the first interval.
3. For each subsequent interval, compare its start with the last merged interval's end:
   - If it overlaps (start ≤ last.end), update the last interval's end to `max(last.end, current.end)`.
   - Else, add the current interval to the result.

```python
def merge_intervals(intervals):
    if not intervals:
        return []
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]
    for start, end in intervals[1:]:
        last_end = merged[-1][1]
        if start <= last_end:  # overlap
            merged[-1][1] = max(last_end, end)
        else:
            merged.append([start, end])
    return merged
```

**Time:** O(n log n) for sorting, O(n) for merging.  
**Space:** O(n) for output.

**Example:**  
`[[1,3],[2,6],[8,10],[15,18]]` → `[[1,6],[8,10],[15,18]]`

---

## 34.4 Insert Interval (LeetCode 57)

**Problem:** Given a set of non‑overlapping intervals sorted by start, insert a new interval into the set (merging if necessary).

**Idea:** Traverse intervals, handling three phases:
- Intervals ending before the new interval starts → add directly.
- Intervals overlapping with the new interval → merge by updating the new interval's start and end.
- Intervals starting after the new interval ends → add the new interval (if not already) and then add remaining intervals.

```python
def insert_interval(intervals, new_interval):
    result = []
    i = 0
    n = len(intervals)
    # Add all intervals ending before new interval starts
    while i < n and intervals[i][1] < new_interval[0]:
        result.append(intervals[i])
        i += 1
    # Merge overlapping intervals
    while i < n and intervals[i][0] <= new_interval[1]:
        new_interval[0] = min(new_interval[0], intervals[i][0])
        new_interval[1] = max(new_interval[1], intervals[i][1])
        i += 1
    result.append(new_interval)
    # Add remaining intervals
    while i < n:
        result.append(intervals[i])
        i += 1
    return result
```

**Time:** O(n) (since intervals are already sorted).  
**Space:** O(n) for output.

**Example:**  
`intervals = [[1,3],[6,9]]`, `new = [2,5]` → `[[1,5],[6,9]]`

---

## 34.5 Meeting Rooms and Calendar Problems

### 34.5.1 Meeting Rooms (LeetCode 252 – Premium)

**Problem:** Given an array of meeting time intervals, determine if a person could attend all meetings (i.e., no overlaps).

**Idea:** Sort by start time and check if any consecutive intervals overlap.

```python
def can_attend_all(intervals):
    intervals.sort(key=lambda x: x[0])
    for i in range(1, len(intervals)):
        if intervals[i][0] < intervals[i-1][1]:
            return False
    return True
```

**Time:** O(n log n) for sorting.

### 34.5.2 Meeting Rooms II (LeetCode 253 – Premium)

**Problem:** Find the minimum number of conference rooms required to accommodate all meetings.

**Idea:** Use a min‑heap to track the earliest ending meeting among currently active rooms. Sort meetings by start time, then for each meeting, if the earliest ending room is free (i.e., its end ≤ current start), reuse it; otherwise allocate a new room. The heap size at the end is the minimum rooms.

```python
import heapq

def min_meeting_rooms(intervals):
    intervals.sort(key=lambda x: x[0])
    rooms = []  # min‑heap of end times
    for start, end in intervals:
        if rooms and rooms[0] <= start:
            heapq.heappop(rooms)  # room becomes free
        heapq.heappush(rooms, end)
    return len(rooms)
```

**Time:** O(n log n) (sorting + heap operations).

**Example:** `[[0,30],[5,10],[15,20]]` → need 2 rooms.

**Alternative: Chronological Ordering (Two Pointers)**
- Create two lists: starts and ends, sort both.
- Use two pointers to simulate events: when a start occurs before an end, we need a new room; otherwise, a room is freed.

```python
def min_meeting_rooms_chronological(intervals):
    starts = sorted([i[0] for i in intervals])
    ends = sorted([i[1] for i in intervals])
    s = e = 0
    rooms = 0
    while s < len(starts):
        if starts[s] < ends[e]:
            rooms += 1
            s += 1
        else:
            e += 1
            s += 1
    return rooms
```

This also runs in O(n log n).

---

## 34.6 Interval Intersection (LeetCode 986)

**Problem:** Given two lists of closed intervals (each sorted by start), find the intersection of the two sets. The intersection of two intervals `[a,b]` and `[c,d]` is `[max(a,c), min(b,d)]` if they overlap.

**Idea:** Use two pointers. For each pair of intervals from A and B, compute the overlap if it exists. Then move the pointer for the interval that ends earlier.

```python
def interval_intersection(A, B):
    i = j = 0
    result = []
    while i < len(A) and j < len(B):
        lo = max(A[i][0], B[j][0])
        hi = min(A[i][1], B[j][1])
        if lo <= hi:
            result.append([lo, hi])
        if A[i][1] < B[j][1]:
            i += 1
        else:
            j += 1
    return result
```

**Time:** O(m + n) where m, n are lengths of A and B.

**Example:**  
A = `[[0,2],[5,10],[13,23],[24,25]]`  
B = `[[1,5],[8,12],[15,24],[25,26]]`  
Intersection: `[[1,2],[5,5],[8,10],[15,23],[24,24],[25,25]]`

---

## 34.7 Non‑Overlapping Intervals (LeetCode 435)

**Problem:** Given a collection of intervals, find the minimum number of intervals you need to remove to make the rest non‑overlapping.

**Idea:** This is the classic "interval scheduling" problem – we want to select as many non‑overlapping intervals as possible, then the answer is total intervals minus that number. The greedy strategy: sort by end time, then always pick the interval that ends earliest and does not conflict with the last chosen.

```python
def erase_overlap_intervals(intervals):
    intervals.sort(key=lambda x: x[1])
    count = 1  # number of intervals we can keep
    last_end = intervals[0][1]
    for i in range(1, len(intervals)):
        if intervals[i][0] >= last_end:
            count += 1
            last_end = intervals[i][1]
    return len(intervals) - count
```

**Time:** O(n log n) for sorting.

**Alternative:** Sort by start and use DP with binary search (but greedy is simpler and optimal here).

---

## 34.8 Minimum Number of Arrows to Burst Balloons (LeetCode 452)

**Problem:** Balloons are represented as intervals [x_start, x_end] (horizontal). An arrow shot vertically at x will burst all balloons where x_start ≤ x ≤ x_end. Find the minimum number of arrows needed to burst all balloons.

**Idea:** Sort balloons by end coordinate. Greedily shoot an arrow at the end of the first balloon, then skip all balloons that are burst by that arrow, and repeat.

```python
def find_min_arrow_shots(points):
    if not points:
        return 0
    points.sort(key=lambda x: x[1])
    arrows = 1
    last_end = points[0][1]
    for start, end in points[1:]:
        if start > last_end:
            arrows += 1
            last_end = end
    return arrows
```

**Time:** O(n log n)

**Example:** `[[10,16],[2,8],[1,6],[7,12]]` → arrows = 2 (e.g., at 6 and 12).

---

## 34.9 Sweep Line Algorithm for Interval Problems

The **sweep line** technique processes events (start and end) in order, maintaining a counter of active intervals. It is useful for problems like:

- Maximum overlapping intervals at any point.
- Finding gaps.
- Interval coverage.

**General idea:**
- For each interval [start, end], create events: (start, +1) and (end, -1) (with care about inclusive/exclusive boundaries).
- Sort events by coordinate (and perhaps by type if needed).
- Sweep through events, updating a counter.

**Example: Find the maximum number of overlapping intervals.**

```python
def max_overlapping(intervals):
    events = []
    for start, end in intervals:
        events.append((start, 1))
        events.append((end, -1))   # if intervals are inclusive, careful at end
    # To handle inclusive correctly, we might treat end as end+1 or use (end, -1) and sort by coordinate then type
    events.sort()
    count = 0
    max_count = 0
    for _, e in events:
        count += e
        max_count = max(max_count, count)
    return max_count
```

**Note:** For inclusive intervals, if one interval ends at x and another starts at x, they should be considered overlapping. To handle this, we can sort events by coordinate, and for ties, process +1 before -1 (or vice versa depending on definition). In the above, we just used integer addition; but if we have (start, +1) and (end, -1), and a start equals an end, the -1 would come first if we sort only by coordinate, which might undercount. Better to sort by (coordinate, type) with a custom order.

---

## 34.10 Employee Free Time (LeetCode 759 – Premium)

**Problem:** Given the schedules of employees (each employee has a list of non‑overlapping intervals), find the common free time intervals (gaps) for all employees.

**Idea:** Flatten all intervals, sort them, merge overlaps, then find the gaps between merged intervals.

```python
def employee_free_time(schedule):
    # schedule is a list of lists of intervals
    intervals = []
    for emp in schedule:
        intervals.extend(emp)
    intervals.sort(key=lambda x: x[0])
    merged = []
    for start, end in intervals:
        if merged and start <= merged[-1][1]:
            merged[-1][1] = max(merged[-1][1], end)
        else:
            merged.append([start, end])
    free = []
    for i in range(1, len(merged)):
        free.append([merged[i-1][1], merged[i][0]])
    return free
```

**Time:** O(N log N) where N is total number of intervals.

---

## 34.11 Summary

```
┌─────────────────────────────────────────────────────────────────────┐
│                    INTERVAL PROBLEMS SUMMARY                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Problem                      │ Technique / Pattern                 │
│───────────────────────────────┼─────────────────────────────────────│
│ Merge Overlapping Intervals   │ Sort by start, then merge.          │
│ Insert Interval               │ Linear scan, three phases.          │
│ Meeting Rooms (attend all)    │ Sort by start, check consecutive.   │
│ Meeting Rooms II (min rooms)  │ Min‑heap of end times or sweep line.│
│ Interval Intersection         │ Two pointers on sorted lists.       │
│ Non‑overlapping Intervals     │ Greedy (sort by end).               │
│ Min Arrows to Burst Balloons  │ Greedy (sort by end).               │
│ Sweep Line                     │ Event‑based counting.              │
│ Employee Free Time             │ Flatten, merge, find gaps.         │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 34.12 Practice Problems

### Easy / Medium
1. **Merge Intervals** (LeetCode 56)
2. **Insert Interval** (LeetCode 57)
3. **Non‑overlapping Intervals** (LeetCode 435)
4. **Minimum Number of Arrows to Burst Balloons** (LeetCode 452)
5. **Meeting Rooms** (LeetCode 252 – Premium)
6. **Meeting Rooms II** (LeetCode 253 – Premium)
7. **Interval List Intersections** (LeetCode 986)
8. **Find the Minimum Number of Conference Rooms Required** (same as Meeting Rooms II)
9. **Can Attend All Meetings** (LeetCode 252)

### Medium / Hard
10. **Employee Free Time** (LeetCode 759 – Premium)
11. **Data Stream as Disjoint Intervals** (LeetCode 352)
12. **Summary Ranges** (LeetCode 228)
13. **Remove Covered Intervals** (LeetCode 1288)
14. **Interval List Unions** (not on LeetCode but similar to merge)
15. **Maximum Overlap Intervals** (sweep line)
16. **Partition Labels** (LeetCode 763) – intervals from last occurrence.

---

## 34.13 Further Reading

1. **"Cracking the Coding Interview"** – Chapter on sorting and searching includes interval problems.
2. **"Elements of Programming Interviews"** – Has a section on intervals.
3. **LeetCode Explore – Intervals** card.
4. **"Algorithm Design"** by Kleinberg & Tardos – Greedy algorithms chapter.

---

> **Coming in Chapter 35**: **Choosing the Right Data Structure** – We'll discuss trade‑offs and how to select appropriate data structures based on problem constraints.

---

**End of Chapter 34**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='33. modified_binary_search.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../10. system_design_real_world_applications/35. choosing_the_right_data_structure.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
